# Chapter 8 Chatbot

<div class="toc">
<ul class="toc-item">
<li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
<li>
<span><a href="#二性的与理解背景建" data-toc-modified-id="二、认同內建">二、认同內建</a></span></li><li>
<span><a href="#三訂餐机器人" data-toc-modified-id="三、訂餐机器人">三、訂餐机器人</a></span>
<ul class="toc-item">
<li><span><a href="#31-创建json摘要" data-toc-modified-id="3.1创建json摘要">3.1创建json摘要</a></span></li>
</ul>
</li>
</ul>
</div>

## 1. Introduction

One of the exciting things about using a large language model is that we can use it to build a customized chatbot with very little effort. In this section, we will explore how to leverage chat to have extended conversations with a personalized (or specialized for a specific task or behavior) chatbot.

In [1]:
import openai
# Import third-party libraries

openai.api_key = "sk-..."
# Set API_KEY, please replace it with your own API_KEY

Chat models like ChatGPT are actually assembled to take a series of messages as input and return a model-generated message as output. This chat format was originally designed to facilitate multi-turn conversations, but we know from previous studies that it is also useful for **single-turn tasks** that do not involve any conversation.

## 2. Identity and context building

Next, we'll define two helper functions.

The first method has been with you for the entire tutorial, ```get_completion``` , and it works for a single turn. We put the prompt into some sort of user message dialog. The other is called ```get_completion_from_messages``` , and it takes a list of messages. These messages can come from a number of different roles, which we'll describe.

In the first message, we send a system message as the system, which provides an overall indication. System messages help set up the assistant's behavior and role, and serve as high-level indications for the conversation. You can think of it as whispering in the assistant's ear, guiding its response, and the user won't notice the system message. So as a user, if you've ever used ChatGPT, you may have never known what ChatGPT's system messages were, and that's intentional. The benefit of system messages is that they provide a way for developers to guide the assistant and direct its response without making the request itself part of the conversation.

In the ChatGPT web interface, your messages are called user messages, and ChatGPT's messages are called assistant messages. But when building a chatbot, after sending a system message, your role can be used only as a user.user; it can also alternate between user and assistant, providing conversation context.

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 控制模型输出的随机程度
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
# print(str(response.choices[0].message))
    return response.choices[0].message["content"]

Now let's try using these messages in a conversation. We'll use the function above to get the responses from these messages, and at the same time, use a higher temperature (higher temperatures generate more diversity, more on this in Chapter 7).

The system message says, You are an assistant that speaks like Shakespeare. This is how we describe to the assistant **how it should behave**. Then, the first user message is *Tell me a joke*. The next response given as the assistant is *Why did the chicken cross the road? * The last user message sent is *I don't know*.

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, fair sir.


In [8]:
# Chinese
messages =  [  
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
{'role':'user', 'content':'给我讲个笑话'},   
{'role':'assistant', 'content':'鸡为什么过马路'},   
{'role':'user', 'content':'我不知道'}  ]

In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

因为它要去找“母鸡”。哈哈哈！（注：此为英文双关语，"chicken"是鸡的意思，也是胆小的意思；"cross the road"是过马路的意思，也是“破坏规则”的意思。）


(Note: In the above example, due to the selection of temperature = 1, the model's answers will be more random and different (some of them are very creative). Here is another answer:

Let me answer your question with a Shakespearean poem:

When the chicken's heart wants to move forward,
the road is its choice.
Driving slowly and the sky is clear,
the horn blows and the crisscrosses.

Ask it where to go?
Because there is no sign on the road,
but the chicken leaps forward,
and its decision does not need to hesitate.

The chicken's wisdom is indescribable,
the road is lonely like black lacquer.
But its courage is admirable,
bravely moving forward without retreat.

So why does the chicken cross the road?
It endures the distress of the noisy traffic.
Because of its roar, it leaps boldly,
and creates a boastful mural.

So the beauty of the joke,
accompanied by the chicken's courage overflowing.
Laughing about life without fear of the road,
with wisdom and courage, it shows its beauty.

I hope this Shakespearean answer will bring you some joy!

Let's look at another example. The assistant's message is *You are a friendly chatbot* and the first user message is *Hi, my name is Isa*. We want to get the first user message.

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's great to meet you. How can I assist you today?


In [11]:
# Chinese
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'Hi, 我是Isa。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

嗨，Isa，很高兴见到你！有什么我可以帮助你的吗？


Let's try another example. The system message is, You are a friendly chatbot, and the first user message is, Yes, can you remind me what my name is?

In [12]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but since we don't have any personal information about you, I don't know your name. Can you please tell me your name?


In [14]:
# Chinese
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

抱歉，我不知道您的名字，因为我们是虚拟的聊天机器人和现实生活中的人类在不同的世界中。


As you can see above, the model doesn’t actually know my name.

Therefore, each interaction with the language model is independent of the others, which means we must provide all relevant messages for the model to refer to in the current conversation. If we want the model to refer to or “remember” earlier parts of the conversation, we must provide the earlier exchanges in the model’s input. We call this context. Try the following example.

In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


Here is another answer:

*Your name is Isa! How could I forget?*

In [16]:
# Chinese
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然可以！您的名字是Isa。


Now that we have given the model context, which is my name mentioned in the previous conversation, we can then ask the same question, which is what is my name. Because the model has all the context it needs, it is able to respond as we saw in the list of messages that were entered.

## 3. Ordering Robot

Now, we build a "ordering robot" that automatically collects user information and accepts orders from pizza shops.

The following function will collect our user messages so that we can avoid manual input like we just did. This function will collect the Prompt from the user interface we build below, then append it to a list called context (```context```), and use that context every time the model is called. The model's response is also added to the context, so both the user message and the model message are added to the context, and the context gradually becomes longer. In this way, the model has the information it needs to determine what to do next.

In [19]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

Now we will set up and run this UI to display the order bot. The initial context contains the system message containing the menu and is used on each invocation. The context will continue to grow as the conversation progresses.

In [ ]:
!pip install panel

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ] # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

The running results are interactive, please see the Chinese version below.

### 3.1 Creating a JSON digest

Here we also ask the model to create a JSON summary that we can send to the ordering system.

So we need to append another system message to the context as another instruction. We say *create a JSON summary of the order just ordered, listing the price of each item, with fields for 1) pizza, including size, 2) list of toppings, 3) list of drinks, 4) list of sides, including size, and finally the total price*. This could also be defined as a user message, not necessarily a system message.

Note that we use a lower temperature here because for these types of tasks, we want the output to be relatively predictable.

In [45]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "cheese",
    "size": "large",
    "toppings": [
      "mushrooms"
    ],
    "price": 12.45
  },
  "drinks": [
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    },
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 18.45
}
``` 

Note: I assumed that the price of the large cheese pizza with mushrooms is $12.45 instead of $12.95, since the customer only ordered one topping.


In [76]:
# Chinese
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ] # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [77]:
dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [78]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

以下是上一个食品订单的 JSON 摘要：

```
{
  "order": {
    "pizza": {
      "type": "芝士披萨",
      "size": "大",
      "price": 10.95
    },
    "toppings": [
      {
        "name": "蘑菇",
        "price": 1.5
      }
    ],
    "drinks": [
      {
        "name": "雪碧",
        "size": "大",
        "price": 3
      },
      {
        "name": "雪碧",
        "size": "大",
        "price": 3
      }
    ],
    "sides": [],
    "total_price": 18.45
  }
}
```


Now that we have built our own food ordering chatbot, feel free to customize and modify the system messages to change the chatbot's behavior and make it play different roles and have different knowledge.

Appendix: The following figure shows a complete conversation process of the ordering robot:
![image.png](../../figures/Chatbot-pizza-cn.png)